In [1]:
import json, requests
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import *

#criando um objeto sparksession object e um appName 
spark=SparkSession.builder.master("local[1]")\
        .appName("Football")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mssql-jdbc-7.4.1.jre8.jar" ) \
        .getOrCreate()

uri = 'https://api.football-data.org/v4/competitions'
headers = { 'X-Auth-Token': '3a48f133afd545dea4f0f25b5f42f400' }

response = requests.get(uri, headers=headers)
rdd = spark.sparkContext.parallelize([response.text])

df = spark.read.option("multiline", "true").json(rdd)
df1 = df.select(explode(df.competitions.area).alias("area"))
df1 = df1.select(to_json(df1.area).alias("area"))
   

schema = StructType(
  [
    StructField('code', StringType(), True),
    StructField('flag', StringType(), True),
    StructField('id', StringType(), True),
    StructField('name', StringType(), True)
  ]
)

df1 = df1.withColumn("area", from_json("area", schema))
df1 = df1.select(col("area.*"))

df1.createOrReplaceTempView("tb_campeonatos")

# currentSeason
df2 = df.select(explode(df.competitions.currentSeason).alias("currentSeason"))
df2 = df2.select(to_json(df2.currentSeason).alias("currentSeason"))

schm02 = StructType(
  [
    StructField("currentMatchday", StringType(), True),
    StructField("endDate", StringType(), True),
    StructField("id", StringType(), True),
    StructField("startDate", StringType(), True),
    StructField("winner", StringType(), True)

  ]  

)

df2 = df2.withColumn("currentSeason", from_json("currentSeason", schm02))
df2 = df2.select(col("currentSeason.*"))

# competitions
df3 = df.select(explode(df.competitions).alias("competitions"))
df3 = df3.select(df3.competitions.emblem, \
                 df3.competitions.id, \
                 df3.competitions.lastUpdated, \
                 df3.competitions.lastUpdated, \
                 df3.competitions.numberOfAvailableSeasons, \
                 df3.competitions.plan, \
                 df3.competitions.type)

df4 = df.select(col("competitions.name").alias("nome"))


23/05/06 15:27:37 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
23/05/06 15:27:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/06 15:27:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/05/06 15:27:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
from datetime import *
dt_inicio = date(2009, 1, 1)
dt_fim = date(2009, 9, 30)

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# dt_fim = datetime('2009-07-30')
df = spark.read.option("header",True).option("delimiter", ",").csv("/Users/eduardoalberto/LoadFile/raw/races.csv")
for a in daterange(dt_inicio, dt_fim):
    t = df.where(df.date >= a)

    t.show()








+------+----+-----+---------+--------------------+----------+--------+--------------------+
|raceId|year|round|circuitId|                name|      date|    time|                 url|
+------+----+-----+---------+--------------------+----------+--------+--------------------+
|     1|2009|    1|        1|Australian Grand ...|2009-03-29|06:00:00|http://en.wikiped...|
|     2|2009|    2|        2|Malaysian Grand Prix|2009-04-05|09:00:00|http://en.wikiped...|
|     3|2009|    3|       17|  Chinese Grand Prix|2009-04-19|07:00:00|http://en.wikiped...|
|     4|2009|    4|        3|  Bahrain Grand Prix|2009-04-26|12:00:00|http://en.wikiped...|
|     5|2009|    5|        4|  Spanish Grand Prix|2009-05-10|12:00:00|http://en.wikiped...|
|     6|2009|    6|        6|   Monaco Grand Prix|2009-05-24|12:00:00|http://en.wikiped...|
|     7|2009|    7|        5|  Turkish Grand Prix|2009-06-07|12:00:00|http://en.wikiped...|
|     8|2009|    8|        9|  British Grand Prix|2009-06-21|12:00:00|http://en.